In [15]:
import ROOT
from ROOT import TFile, TTree, TLorentzVector, TMVA, TCut

import numpy as np
from array import array

In [16]:
def get_values(ev_tree, ef, hf, em, jb):
    m_e = 0.000511 # GeV
    m_pi = 0.139570 # GeV
    m_k = 0.493677 # GeV
    m_p = 0.93827 # GeV
    m_mu = 0.105658 # GeV

    Ee = 10 # GeV
    Ep = 100 # GeV

    pdg = ev_tree.GetLeaf('GeneratedParticles.PDG')
    px = ev_tree.GetLeaf('GeneratedParticles.momentum.x')
    py = ev_tree.GetLeaf('GeneratedParticles.momentum.y')
    pz = ev_tree.GetLeaf('GeneratedParticles.momentum.z')
    E = ev_tree.GetLeaf('GeneratedParticles.energy')

    four_p = TLorentzVector()
    
    Epzh = 0
    pxh = 0
    pyh = 0

    for i in range(px.GetLen()):
        # Values from electron final state
        four_p.SetXYZM(px.GetValue(i),
                       py.GetValue(i),
                       pz.GetValue(i),
                       m_e)
        
        if abs(int(pdg.GetValue(i))) == 11 and E.GetValue(i) > 0.1:
            theta = 3.1415926535 - four_p.Theta()
            ef[0] = four_p.Eta()
            ef[1] = E.GetValue(i)

            Q2 = 2 * Ee * E.GetValue(i) * (1 - np.cos(theta))
            s = 4 * Ee * Ep
            y = 1 - (E.GetValue(i) / Ee) * ((np.cos(theta / 2)) ** 2)
            x = Q2 / (s * y)

            # Clamping the y
            if y >= 0.01 and y <= 0.95:                
                e_x = x
                e_Q2 = Q2

        # Values from the hadronic final state
        four_p.SetPxPyPzE(px.GetValue(i),
                          py.GetValue(i),
                          pz.GetValue(i),
                          E.GetValue(i))

        clamp = four_p.Eta() > -4 and four_p.Eta() < 4

        if abs(int(pdg.GetValue(i))) != 11 and clamp:
            Epzh += E.GetValue(i) - pz.GetValue(i)
            pxh += px.GetValue(i)
            pyh += py.GetValue(i)

    em[0] = x
    em[1] = Q2

    pth2 = np.power(pxh, 2) + np.power(pyh, 2)

    hf[0] = Epzh
    hf[1] = pth2

    y = Epzh / (2 * Ee)
    Q2 = pth2 / (1 - y)
    s = 4 * Ee * Ep
    x = Q2 / (s * y)

    jb[0] = x
    jb[1] = Q2

    return ef, hf, em, jb

In [17]:
filenames = [
    'pythia8NCDIS_10x100_minQ2=100_beamEffects_xAngle=-0.025_hiDiv_vtxfix_1_000.0001.eicrecon.tree.edm4eic.root',
    'pythia8NCDIS_10x100_minQ2=100_beamEffects_xAngle=-0.025_hiDiv_vtxfix_1_000.0002.eicrecon.tree.edm4eic.root',
    'pythia8NCDIS_10x100_minQ2=100_beamEffects_xAngle=-0.025_hiDiv_vtxfix_1_000.0003.eicrecon.tree.edm4eic.root',
    'pythia8NCDIS_10x100_minQ2=100_beamEffects_xAngle=-0.025_hiDiv_vtxfix_1_001.0004.eicrecon.tree.edm4eic.root'
]

In [18]:
dis_file = TFile.Open('dis_values.root', 'RECREATE')
dis_tree = TTree('dis_tree', 'Deep Inelastic Scattering Values')

In [19]:
electron_f = array('f', [ 0, 0 ])
dis_tree.Branch('electron_final_state', electron_f, 'eta/F:final_energy/F')

hadron_f = array('f', [ 0, 0 ])
dis_tree.Branch('hadron_final_state', hadron_f, 'Epz/F:squared_pt/F')

em_outputs = array('f', [ 0, 0 ])
dis_tree.Branch('electron_method_outputs', em_outputs, 'x/F:Q_squared/F')

jb_outputs = array('f', [ 0, 0 ])
dis_tree.Branch('jacquet_blondel_method_outputs', jb_outputs, 'x/F:Q_squared/F')

In [20]:
for filename in filenames:  
    events_file = TFile.Open(filename, 'READ')
    events_tree = events_file.Get('events')
    
    for ev in events_tree:
        electron_f, hadron_f, em_outputs, jb_outputs = get_values(events_tree, electron_f, hadron_f, em_outputs, jb_outputs)
        dis_tree.Fill()

dis_file.cd()
dis_tree.Write()

dis_file.Close()